In [2]:
from numpy import array
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
import datetime

2022-09-12 20:49:29.786704: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-09-12 20:49:29.786768: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [11]:
class CNN_model():

    def __init__(self):
        pass

    def get_data(self, file_location):
        ''' retrieves data from csv returns dataframe'''
        return pd.read_excel(file_location)

    def seperate_ratios(self, ratios_df):
        '''Seperates the ratios into lists and returns a library of the lists with column name as the key and list as value'''
        df = ratios_df.copy()
#         df.set_index('Date', inplace=True)
        ratios_dict = {}
        for columns in ratios_df:
            ratios_dict[columns] = ratios_df[columns].to_list()

        return ratios_dict

    # split a univariate sequence into samples
    def split_sequence(self, sequence, n_steps_in, n_steps_out):
        X, y = list(), list()
        for i in range(len(sequence)):
            # find the end of this pattern
            end_ix = i + n_steps_in
            out_end_ix = end_ix + n_steps_out
            # check if we are beyond the sequence
            if out_end_ix > len(sequence):
                break
            # gather input and output parts of the pattern
            seq_x, seq_y = sequence[i:end_ix], sequence[end_ix:out_end_ix]
            X.append(seq_x)
            y.append(seq_y)
        return array(X), array(y)

    def test_train_splits(self, X, y):
        X_train = X[0:X.shape[0] - 1]
        X_test = X[X.shape[0] - 1:]
        X_train.shape
        X_test.shape
        y_train = y[0:y.shape[0] - 1]
        y_test = y[y.shape[0] - 1:]

        return X_train, y_train, X_test, y_test

    def reshape(self, X_test, X_train, timesteps, features):
        X_test_reshaped = X_test.reshape(
            (X_test.shape[0], timesteps, features))
        X_train_reshaped = X_train.reshape(
            (X_train.shape[0], timesteps, features))
        return X_test_reshaped, X_train_reshaped

    def create_model(self, input_shape, output):
        model = Sequential()
        model.add(
            Conv1D(filters=64,
                   kernel_size=2,
                   activation='relu',
                   input_shape=(input_shape, 1)))
        model.add(MaxPooling1D(pool_size=2))
        model.add(
            Conv1D(filters=32,
                   kernel_size=2,
                   activation='relu',
                   input_shape=(input_shape, 1)))
        model.add(MaxPooling1D(pool_size=2))
        model.add(Flatten())
        model.add(Dense(50, activation='relu'))
        model.add(Dense(output))
        model.compile(optimizer='adam', loss='mse')
        return model

    def fit_model(self, model, X_train, y_train, epochs):
        model.fit(X_train, y_train, epochs=epochs, verbose=0)

    def make_prediction(self, fitted_model, X_test):
        return fitted_model.predict(X_test, verbose=0)

In [131]:
CNN = CNN_model()
##Get Data
df = CNN.get_data('../raw_data/ratios.xlsx')
actual_start_date = (df['Date'].iloc[-1] +
                     datetime.timedelta(days=1)).strftime("%Y-%m-%d")
actual_end_date = (df['Date'].iloc[-1] +
                   datetime.timedelta(days=30)).strftime("%Y-%m-%d")

actual_predictions_cnn = pd.DataFrame()
actual_predictions_cnn['Date'] = pd.date_range(actual_start_date,
                                               actual_end_date)

actual_predictions_cnn

,Date
0,2022-09-09
1,2022-09-10
2,2022-09-11
3,2022-09-12
4,2022-09-13
5,2022-09-14
6,2022-09-15
7,2022-09-16
8,2022-09-17
9,2022-09-18


In [119]:
all_predictions_cnn

,Date


In [96]:
model = CNN.create_model(200, 30)
columns = 'ROST_MMM'
# for columns in df:
X, y = CNN.split_sequence(seperate_ratios[columns], 200, 30)
X_train, y_train, X_test, y_test = CNN.test_train_splits(X, y)
X_test_reshaped, X_train_reshaped = CNN.reshape(X_test, X_train, 200, 1)


test = np.array(seperate_ratios[columns][-200:])
test= test.reshape(1,200,1)
test

array([[[0.62907052],
        [0.65390427],
        [0.66109886],
        [0.63718961],
        [0.64414101],
        [0.65222432],
        [0.62445998],
        [0.66015902],
        [0.6490673 ],
        [0.64814326],
        [0.65295423],
        [0.65501957],
        [0.65191589],
        [0.64954518],
        [0.64035504],
        [0.63937524],
        [0.63099675],
        [0.62461604],
        [0.63034146],
        [0.62499755],
        [0.63984162],
        [0.64371322],
        [0.64172389],
        [0.64818918],
        [0.64355425],
        [0.64734662],
        [0.65566098],
        [0.65575528],
        [0.64892665],
        [0.63996135],
        [0.62272435],
        [0.62667725],
        [0.59468085],
        [0.59245132],
        [0.60895187],
        [0.59899396],
        [0.58970176],
        [0.58319577],
        [0.56720588],
        [0.56554233],
        [0.5721447 ],
        [0.56338622],
        [0.57953104],
        [0.56674165],
        [0.57050048],
        [0

In [100]:
CNN.fit_model(model, X_train_reshaped, y_train, 1000)

In [103]:
predictions = CNN.make_prediction(model,test)

In [104]:
predictions

array([[0.7814273 , 0.78934866, 0.78488463, 0.7811894 , 0.7837341 ,
        0.77951294, 0.7818634 , 0.7761598 , 0.76992315, 0.76916707,
        0.756387  , 0.7488687 , 0.73094344, 0.72224426, 0.70128405,
        0.6999623 , 0.6928433 , 0.66733944, 0.6519996 , 0.62858313,
        0.62960255, 0.61524445, 0.6155769 , 0.57139754, 0.56790525,
        0.56220007, 0.5621038 , 0.56484187, 0.5545343 , 0.5451977 ]],
      dtype=float32)

In [99]:
y_test

array([[0.58379529, 0.57111421, 0.57793946, 0.56696143, 0.59341222,
        0.56928981, 0.57524167, 0.57841885, 0.57513471, 0.58079711,
        0.59296224, 0.59594044, 0.59939409, 0.62067697, 0.63382032,
        0.63143321, 0.62459537, 0.62011019, 0.61835573, 0.63983852,
        0.63457272, 0.67333424, 0.68534818, 0.68899274, 0.6913377 ,
        0.70206166, 0.71837237, 0.7437393 , 0.76433013, 0.77445697]])

In [28]:
df = pd.read_csv('../raw_data/CNN_preds.csv')

In [29]:
df

,Date,CMG_ETN,ROST_MMM,RE_ZTS,MCK_GLW,D_WBD,DVA_OMC,CARR_GOOGL,PGR_CTSH,RJF_MSFT,PKG_FIS
0,2022-09-09,10.660976,0.635300,1.365520,4.946893,2.325745,1.607635,0.358417,1.331514,0.331920,1.299773
1,2022-09-10,10.511396,0.628278,1.351953,4.982984,2.312927,1.603552,0.358283,1.332626,0.332074,1.301620
2,2022-09-11,10.501227,0.624439,1.339238,4.988275,2.320449,1.593868,0.357903,1.333789,0.332127,1.303265
3,2022-09-12,10.858273,0.621429,1.332047,5.023298,2.300386,1.594499,0.357967,1.334840,0.332233,1.304616
4,2022-09-13,10.690048,0.627996,1.327158,5.057480,2.289474,1.594923,0.357913,1.335710,0.332390,1.306189
5,2022-09-14,10.702137,0.628466,1.298375,5.062571,2.343561,1.594776,0.357653,1.336723,0.332722,1.307621
6,2022-09-15,11.066820,0.631312,1.271542,5.099197,2.354880,1.606974,0.357718,1.337725,0.332781,1.309264
7,2022-09-16,11.032448,0.628108,1.244844,5.123846,2.418091,1.622320,0.357586,1.338862,0.332895,1.311063
8,2022-09-17,10.941701,0.616239,1.245530,5.145099,2.422765,1.633061,0.357474,1.340066,0.333105,1.312798
9,2022-09-18,11.185501,0.622907,1.223893,5.159016,2.433698,1.644302,0.357486,1.341191,0.333321,1.314545


In [34]:
def mean_absolute_percentage_error(y_true, predictions):
    '''Calculates MAPE for predictions'''
    y_true, predictions = np.array(y_true), np.array(predictions)
    return np.mean(np.abs((y_true - predictions) / y_true)) * 100


mean_absolute_percentage_error(y_test,predictions)

8.709868843851186

In [136]:
df = pd.read_csv('../raw_data/CNN_preds.csv')

In [137]:
df

,Date,CMG_ETN,ROST_MMM,RE_ZTS,MCK_GLW,D_WBD,DVA_OMC,CARR_GOOGL,PGR_CTSH,RJF_MSFT,PKG_FIS
0,2022-09-09,10.660976,0.635300,1.365520,4.946893,2.325745,1.607635,0.358417,1.331514,0.331920,1.299773
1,2022-09-10,10.511396,0.628278,1.351953,4.982984,2.312927,1.603552,0.358283,1.332626,0.332074,1.301620
2,2022-09-11,10.501227,0.624439,1.339238,4.988275,2.320449,1.593868,0.357903,1.333789,0.332127,1.303265
3,2022-09-12,10.858273,0.621429,1.332047,5.023298,2.300386,1.594499,0.357967,1.334840,0.332233,1.304616
4,2022-09-13,10.690048,0.627996,1.327158,5.057480,2.289474,1.594923,0.357913,1.335710,0.332390,1.306189
5,2022-09-14,10.702137,0.628466,1.298375,5.062571,2.343561,1.594776,0.357653,1.336723,0.332722,1.307621
6,2022-09-15,11.066820,0.631312,1.271542,5.099197,2.354880,1.606974,0.357718,1.337725,0.332781,1.309264
7,2022-09-16,11.032448,0.628108,1.244844,5.123846,2.418091,1.622320,0.357586,1.338862,0.332895,1.311063
8,2022-09-17,10.941701,0.616239,1.245530,5.145099,2.422765,1.633061,0.357474,1.340066,0.333105,1.312798
9,2022-09-18,11.185501,0.622907,1.223893,5.159016,2.433698,1.644302,0.357486,1.341191,0.333321,1.314545


In [133]:
mean_absolute_percentage_error(y_test,df['predictions)

22.237656748317825